# Train a Convolutional Variational Autoencoder 

Using the Conv-VAE package to train a convolutional variational autoencoder. The parameters of the training are describe by an experiment run of type "conv_vae". 

The result of runing the code in this notebook is the model files that are stored in the experiment directory. As the model files 

__Note:__ The result of running this code is the creation of a "final model". The encoder portion of this model is the component that can be used for the encoder of the robot. Once the training had been done, the subdir of this experiment and the checkpoint we want to use will need to updated into the experiment run-specific file in the model_subdir and model_checkpoint fields.


In [ ]:
import sys
sys.path.append("..")
from settings import Config
import pathlib
from pprint import pprint
import shutil

# adding the Julian-8897-Conv-VAE-PyTorch into the path
sys.path.append(Config()["conv_vae"]["code_dir"])

# Oh, this hack was fixing something, but for me it is the other way around
#temp = pathlib.PosixPath
#pathlib.PosixPath = pathlib.WindowsPath

from conv_vae import get_conv_vae_config, create_configured_vae_json, train,  latest_training_run

In [ ]:
# If it is set to true, no actual copying will be done
dry_run = False

# Specify and load the experiment
experiment = "conv_vae"
run = "vae_02" # was vae_01
exp = Config().get_experiment(experiment, run)
pprint(exp)

### Create the training data

The training data is the set of pictures from a demonstration. The following data copies the specific pictures and puts them the training directory specific to the run.

In [9]:
def copy_images_to_training_dir(taskname, training_image_dir):
    """Copy all the images from a specific task into the training image dir."""
    task_dir = pathlib.Path(demos_dir, taskname)
    # _, task_dir = ui_choose_task(offer_task_creation=True)

    for demo in task_dir.iterdir():
        if not demo.is_dir(): continue
        for item in demo.iterdir():
            if item.suffix != ".jpg": continue
            name = f"{demo.name}_{item.stem}.jpg"
            destination = pathlib.Path(training_image_dir, name)
            print(f"copy {item} to \n{destination}")
            if not dry_run:
                shutil.copyfile(item, destination)

In [ ]:
demos_top = pathlib.Path(Config()["demos"]["directory"])
demos_dir = pathlib.Path(demos_top, "demos")

subdir_count = sum(1 for item in demos_dir.iterdir() if item.is_dir())
print(f"Number of demo directories: {subdir_count}")

# Deciding on the location of the training data
training_data_dir = pathlib.Path(exp["data_dir"], exp["training_data_dir"])
# training_data_dir = pathlib.Path(Config()["conv_vae"]["training_data_dir"])
training_image_dir = pathlib.Path(training_data_dir, "Images")
training_image_dir.mkdir(exist_ok = False, parents=True)

print(f"Training data dir={training_image_dir}")

# Define a set of common image file extensions
image_extensions = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp"}
# Count the image files
image_count = sum(1 for item in training_image_dir.iterdir() if item.suffix.lower() in image_extensions and item.is_file())

print(f"Number of image files in training dir: {image_count}")

if image_count == 0:
    taskname = exp['training_task']
    copy_images_to_training_dir(
        taskname = taskname, training_image_dir=training_image_dir)
else:
    print("There are already images in training image dir {training_image_dir}. Do not repeat the copying.")            


# Train a ConvVAE on the training data

__Important__ After the training finished, in order to use the resulting system, one need to edit the run file (eg: vae_01.yaml) and enter into it the subdirectory of the model (eg. "0108_203019") and the name of the checkpoint we want to use. These might also be better put into the machine specific directory.

```
model_subdir: '1222_150723'
model_checkpoint: 'checkpoint-epoch100.pth'
```

In [11]:
# Create the vae configuration, based on the experiment
file = create_configured_vae_json(exp)
print(file)
vae_config = get_conv_vae_config(file)

# actually run the training
print(f'Running the trainer from scratch for {vae_config["trainer"]["epochs"]}')
trainer = train(vae_config)

KeyboardInterrupt: 

In [ ]:
# These are the metrics recorded
# they are of utils/util.py / MetricTracker which has a pandas dataframe as data
print(trainer.train_metrics)
print(trainer.valid_metrics)
# 
trainer.train_metrics._data
# trainer.valid_metrics._data